In [1]:
%matplotlib inline

In [2]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as ndimage
import xarray as xr
import metpy
from netCDF4 import Dataset

from metpy.units import units
import metpy.calc as mpcalc
import scipy.ndimage as ndimage

In [3]:
crs = ccrs.LambertConformal(central_longitude=-57.5, central_latitude=-12.5)

In [4]:
def plot_background(ax, extent):
    ax.set_extent(extent)
    ax.add_feature(cfeature.COASTLINE.with_scale('100m'), linewidth=0.5)
    ax.add_feature(cfeature.STATES, linewidth=0.5)
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    return ax

In [5]:
ds = Dataset("../Data/REANALYSIS/ERA5/era5_20170314.nc")
print(ds)

<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2018-08-01 10:58:18 GMT by grib_to_netcdf-2.8.0: grib_to_netcdf /data/data05/scratch/12/d4/_mars-atls18-a82bacafb5c306db76464bc7e824bb75-dtWMJ7.grib -o /data/data03/scratch/5b/e4/_grib2netcdf-atls19-98f536083ae965b31b0d04811be6f4c6-r0HjMW.nc -utime
    dimensions(sizes): longitude(1440), latitude(721), level(10), time(24)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 level(level), int32 time(time), int16 pv(time,level,latitude,longitude), int16 crwc(time,level,latitude,longitude), int16 cswc(time,level,latitude,longitude), int16 z(time,level,latitude,longitude), int16 t(time,level,latitude,longitude), int16 q(time,level,latitude,longitude), int16 w(time,level,latitude,longitude), int16 vo(time,level,latitude,longitude), int16 d(time,level,latitude,longitude), int16 u(time,level,latitude,longitude), int16 v

In [6]:
t = ds.variables['t'][:].squeeze() * units.kelvin
u = ds.variables['u'][:].squeeze() * units('m/s')
v = ds.variables['v'][:].squeeze() * units('m/s')
z = ds.variables['z'][:].squeeze() * units.meter
lat = ds.variables['latitude'][:].squeeze() #* units.degree
lon = (ds.variables['longitude'][:].squeeze() - 180) #* units.degree

lon_2d, lat_2d = np.meshgrid(lon, lat)

In [12]:
print(t[12][5][0])
print(t[12][5][1])

[243.0144535 243.0144535 243.0144535 ... 243.0144535 243.0144535 243.0144535] kelvin
[243.73685589 243.74301273 243.74916957 ... 243.71838538 243.72454222 243.73069906] kelvin


In [ ]:
lon

In [ ]:
type(dy)

In [ ]:
type(dx)

In [17]:
#-- Calculating fields
dx, dy = mpcalc.lat_lon_grid_deltas(lon, lat)
dy *= -1

adv_t_850 = mpcalc.advection(t[12][5], [u[12][5], v[12][5]], (dx, dy), dim_order='yx')
adv_t_850 = adv_t_850[:][~np.isnan(adv_t_850)]
z_850 = ndimage.gaussian_filter(z[12][5], sigma=3, order=0) * units.meter
adv_t_850 = ndimage.gaussian_filter(adv_t_850, sigma=3, order=0) * units('K/sec')
print(z_850)
print(adv_t_850)

[[11215.40972935 11215.43917617 11215.49490517 ... 11214.15041772  11214.33724358 11214.43561212] [11223.24637914 11223.27814549 11223.3386902  ... 11221.92649525  11222.12233571 11222.22574072] [11237.69671422 11237.73184425 11237.79953036 ... 11236.28919962  11236.49939828 11236.61085224] ... [12455.20696418 12455.18785065 12455.1487182  ... 12455.38851108  12455.37213359 12455.36544984] [12452.41854826 12452.4032877  12452.37096084 ... 12452.53221958  12452.51259044 12452.50464874] [12450.70710122 12450.69328726 12450.66342385 ... 12450.79717087  12450.77437409 12450.76502088]] meter
[-5.57761075e-04 -5.56510775e-04 -5.54427971e-04 ...  2.32248937e-05  2.63580835e-05  2.82039653e-05] kelvin / second


In [ ]:
extent_sa = [-85., -60., -30., 15.] #-- [min lon, min lat, max lon, max lat]
fig_type = 'SA'